In [10]:
import json
import folium
import pandas
import requests
from bs4 import BeautifulSoup
import urllib.request
import geotag
import kml2geojson 
import zipfile
from xml.dom import minidom
import os

In [11]:
# https://community.esri.com/docs/DOC-8814-convert-kmz-to-kml
# def kmz_to_kml(fname):
#     """save kmz to kml"""
#     zf = zipfile.ZipFile(fname,'r')
#     for fn in zf.namelist():
#         if fn.endswith('.kml'):
#             content = zf.read(fn)
#             xmldoc = minidom.parseString(content)
#             out_name = (fname.replace(".kmz",".kml")).replace("\\","/")
#             out = open(out_name,'w')
#             out.writelines(xmldoc.toxml())
#             out.close()
#         else:
#             print("no kml file")
# def kmz_to_kml(fname):
#     from zipfile import ZipFile
#     kmz = ZipFile(fname, 'r')
#     kml = kmz.open('doc.kml', 'r') #fname[0:3]+
#     out_name = (fname.replace(".kmz", ".kml"))
    

In [12]:
place_num = 171384
place_file = open("initial-10/places_"+(str)(place_num)+".json")
places = json.load(place_file)

# soup.
# https://gadm.org/maps.html

r = requests.get("https://gadm.org/maps.html")
htmlcontent = BeautifulSoup(r.content, "html.parser")
countries = []
country_containers = htmlcontent.find('div', class_="container-fluid main-container").p.small
# print(country_containers)

country_codes = {}
count = 'A'

for a_tag in country_containers.find_all('a', href=True):
    name = a_tag.string
    if name[0].islower():
        name = count + name
        count = chr(ord(count) + 1)
        if (count == 'X'):
            count = chr(ord(count) + 1)
    country_codes.update({name: (a_tag['href'][5:8])})


In [15]:
def get_geojson(code):
    url="https://biogeo.ucdavis.edu/data/gadm3.6/kmz/gadm36_" + code + "_0.kmz"
    urllib.request.urlretrieve(url, './kmz-files/' + code + ".kml.zip")
    print(code)
    zf = zipfile.ZipFile('./kmz-files/' + code + ".kml.zip")
    zf.extractall('./kml-files')
    zf.close()
    kml2geojson.main.convert('./kml-files/gadm36_' + code + "_0.kml", "./geojsons/", separate_folders=False, style_type=None, style_filename='style.json')
    

In [16]:
# https://biogeo.ucdavis.edu/data/gadm3.6/kmz/gadm36_AFG_0.kmz
GT = geotag.Geotag()
place_names = []

for place in places:
    tag = place["geotag"]
    if (tag in GT.path2entry):
        tag = GT.path2entry[tag]
        place_names.append(tag['name'])

print(place_names)

for place in place_names:
    if (place in country_codes.keys()):
        country = country_codes.get(place)
        get_geojson(country)
            

['Vienna', 'Austria', 'Germany', 'Neuss', 'Berlin', 'Graz', 'Duesseldorf', 'France', 'Syria', 'Albania', 'Belgium', 'North Rhine-Westphalia', 'Tunisia', 'Paris', 'Salzburg']
AUT
DEU
FRA
SYR
ALB
BEL
TUN


In [ ]:
#   {
#     "story_type": "World Affairs",
#     "story_id": 186150,
#     "story_name": "Venezuela opposition leader",
#     "geotext": "Venezuela",
#     "geotag": "VE"
#   }

def geojson_list():
    geojsons = []
    for item in os.listdir('./geojsons'): 
        file = os.path.dirname(item)
        code = file[-7:-4]
        dict_ =    {"story_type": "World Affairs",
                    "story_id": place_num,
                    "story_name": "Venezuela opposition leader",
                    "geotext": country_codes.keys()[country_codes.values().index(code)],
                    "geotag": code,
                    "geojson_file": file
                   } 
        geojsons.append(dict_)
        
    with open('geojson_list.json', 'w') as f:
        json.dump(geojsons, f)
    